In [ ]:
# !rm -r runs

In [ ]:
import json
import os
import time
import cv2
import numpy as np
from pathlib import Path
from IPython.display import clear_output, display, FileLink
from ipywidgets import Layout, HTML, AppLayout, Label, Checkbox, BoundedIntText, RadioButtons, Button, Layout, ColorPicker, FloatRangeSlider, FloatSlider, IntSlider, Image, ToggleButton, Dropdown, VBox, HBox, FileUpload, Output, Accordion, Tab, GridspecLayout, Text
from utils.utils import *
from utils.yolo import YoloInfer, bbox_iou
from utils.stream import StreamHandler
from utils.tracker import EuclideanDistTracker, CarRecord
# from utils.sort import Sort

expand_boxes(np.zeros((1,4)), (0,0))
is_crossing_stop_line(np.zeros((1,4)), np.arange(4))

BASE_DIR = Path().resolve()

ENGINE_DIR = BASE_DIR / 'engine'
ENGINE_NAME = 'cwpssb_dyn'
CLASSES = ["car", "passenger", "seat belt", "windshield"]

engine = YoloInfer()
st = StreamHandler()
car_tracker = EuclideanDistTracker()
recorder = CarRecord()
records = recorder.records
# car_tracker = Sort()

engine_loaded = False
input_loaded = False

fps = 0
fps_filt = 0
frame_count = -1
frame_id = 0

In [ ]:
def infer(c):
  global tt, fps_filt, frame_count, records, frame_id
  frame_count += 1
  if frame_count%(frame_skip_spn.value+1)!=0:
    tt = time.perf_counter()
    return
  frame = c.new
  img = frame.copy()
  # img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

  # roi shape
  h_img, w_img = img.shape[:2]
  verts = np.vstack((np.hstack([x.value for x in roi_xs])*w_img, np.hstack([y.value for y in roi_ys])*h_img)).T.astype(int)
  verts[[2, 3]] = verts[[3,2]]
  l,t,w,h = cv2.boundingRect(verts)

  # stop line position
  pos = np.hstack((np.array(stop_line_x.value)*w_img, np.array(stop_line_y.value)*h_img)).astype(int)
  pos[[1, 2]] = pos[[2, 1]]

  # get roi image
  mask = np.zeros(img.shape, dtype=np.uint8)
  cv2.fillPoly(mask, [np.int32(verts)], (255,)*img.shape[2])
  masked = cv2.bitwise_and(img, mask)
  roi = masked[t:t+h, l:l+w, :]

  # infer car and windshield
  cw_res = engine.infer(roi, imgsz=480, classes=[0, 3], render=False)

  # check if any object is detected
  if cw_res.xyxy.size>0:
    # restore box coordinate from roi coordinate to image coordinate
    cw_boxes = cw_res.xyxy
    expand_boxes(cw_boxes[:, :4], (l,t))
    cw_res.raw_img = frame
    cw_res.xyxy = cw_boxes

    # split the boxes separately
    cars = cw_res.get_class_boxes(0, 'xyxy')
    ws = cw_res.get_class_boxes(3, 'xyxy')

    # check if it detected the cars
    if cars.size > 0:
      # isolate the bounding box
      cboxes, cconfs, ccls_id = cars[:, :4], cars[:, 4], cars[:, 5]
      # create placeholder columns for other properties (id, stop_line, n_passenger, n_seat_belt)
      cboxes_id = np.zeros((cboxes.shape[0], cboxes.shape[1]+4))
      # get object id (tracker)
      cboxes_id[:, :5] = car_tracker.update(cboxes)
      ws_imgs = []

      # check if it detected the windshields
      if ws.size > 0:
        # isolate the bounding boxes
        wboxes, wconfs, wcls_id = ws[:, :4], ws[:, 4], ws[:, 5]
        # create placeholder column for id
        wboxes_id = -np.ones((wboxes.shape[0], wboxes.shape[1]+1))
        wboxes_id[:, :-1] = wboxes

        # assign the each windshield its car_id (based on iou)
        invalid = []
        for idx, w in enumerate(wboxes_id): #for each windshield bounding box
          # calculate its iou against all car bounding boxes
          car_index = bbox_iou(np.expand_dims(w[:4], 0), cars[:, :4])
          # windshield with no car will be discarded
          if car_index.sum()==0:
            invalid.append(idx)
            continue
          # assign the car_id with highest iou score
          w[-1] = cboxes_id[np.argmax(car_index), 4]
        # discard invalid boxes
        wboxes_id = np.delete(wboxes_id, invalid, 0)

        # seat belt inference
        ws_imgs = []
        for wbox, wconf, wcls_id in zip(wboxes_id, wconfs, wcls_id.astype(int)): # for each windshield bounding box
          box = wbox.astype(int)
          # isolate (crop) windshield from image
          ws = frame[box[1]:box[3], box[0]:box[2], :]
          # do the inference
          ps_res = engine.infer(ws, imgsz=480, classes=[1, 2], render=False)
          ws_imgs.append((int(wbox[-1]), ps_res))
          a = cboxes_id[cboxes_id[:, 4]==wbox[-1]]
          # count the detected seatbelt
          a[0, 7] = ps_res.classids[ps_res.classids==2].size
          a[0, 6] = ps_res.classids[ps_res.classids==1].size
          # append the result for that specific car_id
          cboxes_id[cboxes_id[:, 4]==wbox[-1]] = a

      # check if the cars crossed the stop line when the light is red
      if traffic_light.value=='Red':
        cboxes_id[:, 5] = is_crossing_stop_line(cboxes_id, pos).astype(cboxes_id.dtype)

      # update the result
      records = recorder.update(frame_id, cboxes_id, ws_imgs, cw_res)
      # draw result on image and result table
      rows = ""
      for idx in reversed(list(records.keys())):
        recs = records[idx]
        rows += row_template.format(idx, any(recs['stop_line']), max(recs['n_passenger']), max(recs['n_seat_belt']))
        if idx not in cboxes_id[:, 4]:
          continue
        plot_one_box(recs['positions'][-1], img, color=[0,0,255] if any(recs['stop_line']) else [84,185,29], label=f"id:{idx}, sb:{max(recs['n_seat_belt'])}")
      out_table.value = table_style + out_table_template.format(rows)

  # draw stop line and roi
  draw_stop_line(img, pos, stop_line_thickness.value, hex2bgr("#00ff00" if traffic_light.value=="Green" else "#ff0000"))
  draw_roi(img, verts, roi_thickness.value, hex2bgr(roi_color.value))

  # calculte fps and draw it on image
  frame_id += 1
  ed = time.perf_counter()
  fps = 1/(ed-tt)
  fps_filt = .9*fps_filt + .1*fps
  tt = ed
  img = imresize(img, height=480)
  cv2.putText(img, f"FPS: {fps_filt:#.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
  image_widget.value = bgr8_to_jpeg(img)
  tt = ed

def init_camera(obj):
  global input_loaded
  if input_loaded:
    return
  w, h = cam_res.value
  cw, ch = cap_res.value
  obj.icon = 'fa-circle-o'
  # obj.icon = 'gear spin lg'
  with status_bar:
    st.set_handler(0, height=h, width=w, ee_mode=2, ee_strength=1, tnr_mode=2, tnr_strength=1, contrast=1.5, brightness=-0.2, saturation=1.2, capture_height=ch, capture_width=cw, capture_fps=30)
    print('Camera initialised succesfully')
    input_loaded = True
  obj.icon = 'fa-check-circle-o'
  obj.disabled = True
  cam_release_btn.disabled = False
  reset_tracker()

def release_camera(obj):
  global input_loaded
  if not input_loaded:
    return
  st.release()
  cam_init_btn.icon = 'circle'
  obj.disabled = True
  cam_init_btn.disabled = False
  with status_bar:
    print('Camera released')
    input_loaded = False

def cap_frame(change):
  if not input_loaded:
    return
  st.stream.running = state = change.new
  if state:
    st.stream.observe(cf, names='value')
  else:
    st.stream.unobserve(cf, names='value')

def cf(change):
  img = change.new
  # img = cv2.fastNlMeansDenoisingColored(img, None, 15, 10, 7, 21)
  h, w = img.shape[:2]

  # stop line
  pos = np.hstack((np.array(stop_line_x.value)*w, np.array(stop_line_y.value)*h)).astype(int)
  pos[[1, 2]] = pos[[2, 1]]
  lt = stop_line_thickness.value
  col = hex2bgr("#00ff00" if traffic_light.value=="Green" else "#ff0000")
  draw_stop_line(img, pos, lt, col)

  # roi
  verts = np.vstack((np.hstack([x.value for x in roi_xs])*w, np.hstack([y.value for y in roi_ys])*h)).T.astype(int)
  verts[[2,3]] = verts[[3,2]]
  r_lt = roi_thickness.value
  r_col = hex2bgr(roi_color.value)
  draw_roi(img, verts, r_lt, r_col)
  image_widget.value = bgr8_to_jpeg(img)
  time.sleep(1/30)

def ff(b):
  global input_loaded
  if input_loaded:
    if st.is_camera:
      release_camera(cam_release_btn)
    else:
      st.stream.release()
  for elem in vid_upload.value.values():
    name = elem['metadata']['name']
    print(name)
    with open('vids/'+name, 'wb') as file:
      file.write(elem['content'])
    print("Done")
  vid_upload.style.button_color = "lightgreen"
  st.set_handler(1, filepath='vids/'+name)
  input_loaded = True
  reset_tracker()

def start_infer(c):
  global tt
  st.stream.running = state = c.new
  start_btn.icon = 'stop' if state else 'play'
  start_btn.description = 'Stop' if state else 'Start'
  cap_frame_btn.disabled = state

  if state:
    tt = time.perf_counter()
    with status_bar:
      print('starting...')
      st.stream.observe(infer, names='value')
  else:
    with status_bar:
      print('stopping...')
      st.stream.unobserve(infer, names='value')

def load_engines(obj):
  global engine_loaded
  if engine_loaded:
    return
  obj.icon = 'fa-circle-o'
  size = model_sel.value
  backend = backend_sel.value
  suffix = '.onnx' if backend=='onnx' else '.engine'
  engine_path = ENGINE_DIR / backend / size / (ENGINE_NAME+suffix)
  with status_bar:
    print('loading engine...')
    engine.setup_model(str(engine_path), CLASSES)
    print('warming up the engine...')
    w = engine.infer(np.zeros((480,480,3), dtype=np.uint8), imgsz=480)
    print('engine loaded...')
  obj.icon = 'fa-check-circle-o'
  engine_loaded = True

def draw_stop_line(frame, pos, lt, col):
  x1,y1,x2,y2 = pos
  cv2.line(frame, (x1, y1), (x2, y2), col, thickness=lt)

def draw_roi(frame, verts, lt, col):
  cv2.polylines(frame, [np.int32(verts)], True, col, lt)

def on_iou_change(c):
  engine.iou = c.new

def on_conf_change(c):
  engine.conf = c.new

def on_trk_change(c):
  car_tracker.thres = c.new

def on_save_change(c):
  recorder.nosave = not c.new

def open_url(obj):
  global input_loaded
  url = url_text.value
  obj.icon = 'fa-circle-o'
  with status_bar:
    st.set_handler(2, url=url)
  obj.icon = 'fa-check-circle-o'
  input_loaded = True
  reset_tracker()

def reset_tracker():
  recorder.destroy()
  car_tracker.destroy()
  download_link.clear_output()

def download_results(obj):
  run_dir = recorder.RUN_DIR.relative_to(BASE_DIR)
  download_link.clear_output()
  with download_link:
    print('zipping, please wait...')
  !zip -r {str(run_dir.with_suffix('.zip'))} {str(run_dir)}
  download_link.clear_output()
  link = FileLink(run_dir.with_suffix('.zip'))
  with download_link:
    display(link)

In [ ]:
out_table_template = "<table><tr><th>ID</th><th>Crossed Stop Line?</th><th>Detected Passenger</th><th>Detected Seat Belt</th></tr>{}</table>"
table_style = "<style>table {border-collapse: collapse; width: 100%;}" + \
              "td, th {text-align: center; padding: 8px;}" + \
              "tr:nth-child(odd) {background-color: #dddddd;}</style>"

row_template = "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>"

# output table
out_table = HTML(value=table_style + out_table_template.format(''))
download_button = Button(description='Download Results', disable=False, icon='download')
download_link = Output()
save_img_out = Checkbox(value=True, description='Save output images', disabled=False, indent=False)
out_box = VBox([download_button, download_link, VBox([out_table], layout=Layout(height='500px', overflow_y='auto'))])
download_button.on_click(download_results)
save_img_out.observe(on_save_change, names='value')

# model selection
backend_sel = Dropdown(
  options=[('ONNX', 'onnx'), ('TensorRT', 'trt_poly')],
  description='Backend',
  value='trt_poly')
model_sel = Dropdown(
  options=[('yolov3', 'reg'), ('yolov3-tiny', 'tiny'), ('yolov5s', 'small'), ('yolov5n', 'nano')],
  description='Model',
  value='tiny')
conf = FloatSlider(
  value=0.5, min=0, max=1, step=0.01, description='Confidence',
  disabled=False, continuous_update=False, orientation='horizontal',
  readout=True, readout_format='.2f')
iou = FloatSlider(
  value=0.4, min=0, max=0.99, step=0.01, description='IOU',
  disabled=False, continuous_update=False, orientation='horizontal',
  readout=True, readout_format='.2f',)
tracking_thres = IntSlider(
  value=25, min=1, max=100, step=1, description='Tracking Thres.',
  disabled=False, continuous_update=False, orientation='horizontal',
  readout=True, readout_format='d')
load_engine_btn = Button(description='Load Model', disable=False, icon='fa-circle')
load_engine_btn.on_click(load_engines)
thres_ctrl = VBox([conf, iou, tracking_thres])
model_select = VBox([backend_sel, model_sel, load_engine_btn])
model_setup = VBox([model_select, thres_ctrl])
iou.observe(on_iou_change, names='value')
conf.observe(on_conf_change, names='value')
tracking_thres.observe(on_trk_change, names='value')

# stop line
stop_line_x = FloatRangeSlider(value=[.25, .75], min=0, max=1, step=.001, description='x',
                               disable=False, continuous_update=False, orientation='horizontal',
                               readout=True, readout_format='.3f')
stop_line_y = FloatRangeSlider(value=[.5, .5], min=0, max=1, step=.001, description='y',
                               disable=False, continuous_update=False, orientation='horizontal',
                               readout=True, readout_format='.3f')
stop_line_thickness = IntSlider(value=2, min=1, max=10, description='Thickness', disable=False,
                                continuous_update=False, orientation='horizontal', readout=True)
traffic_light = RadioButtons(options=['Red', 'Green'], value='Green', description='Traffic Light', disabled=False)
# stop_line_color = ColorPicker(concise=False, description='Color', value='#00ff00', disabled=False)
line_ctrl = VBox([stop_line_x, stop_line_y, stop_line_thickness, traffic_light])

# roi
roi_xs = []
roi_ys = []
for idx in range(2):
  roi_xs.append(FloatRangeSlider(value=[.25, .75], min=0, max=1, step=.001,
                                 description=f'x{idx}', disabled=False, continuous_update=False,
                                 orientation='horizontal',
                                 readout=True,readout_format='.3f',))
  roi_ys.append(FloatRangeSlider(value=[.25+idx*.5, .25+idx*.5], min=0, max=1, step=.001,
                                 description=f'y{idx}', disabled=False, continuous_update=False,
                                 orientation='horizontal', readout=True,readout_format='.3f',))
roi_thickness = IntSlider(value=2, min=1, max=10, description='Thickness', disable=False,
                          continuous_update=False, orientation='horizontal',readout=True)
roi_color = ColorPicker(concise=False, description='Color', value='#ffff00', disabled=False)
roi_ctrl = VBox(roi_xs + roi_ys + [roi_thickness, roi_color])

# camera
cam_res = Dropdown(
  options=[('640x360', (640, 360)), ('640x480', (640, 480)), ('854x480', (854, 480)), ('1280x720', (1280, 720)), ('1920x1080', (1920, 1080)), ('2616x1472', (2616, 1472)), ('3840x2160', (3840, 2160))],
  description='Size',
  value=(854, 480))
cap_res = Dropdown(
  options=[('1280x720', (1280, 720)), ('1920x1080', (1920, 1080)), ('2616x1472', (2616, 1472)), ('3840x2160', (3840, 2160))],
  description='Capture Size',
  value=(1280, 720))
cam_init_btn = Button(description='Start Camera', disabled=False, icon='circle')
cam_init_btn.on_click(init_camera)
cam_release_btn = Button(description='Release Camera', disabled=True, icon='square')
cam_release_btn.on_click(release_camera)
cam_setup = VBox([cap_res, cam_res, cam_init_btn, cam_release_btn])

# file
vid_upload = FileUpload(accept='video/*', multiple=False)
vid_upload.observe(ff, names='_counter')
vid_up_box = VBox([Label('Max. size: 200MB'), vid_upload])

# url
url_text = Text(value='https://youtu.be/MNn9qKG2UFI', placeholder='rtmp:// or http://', description='URL', disabled=False)
url_btn = Button(description='Open URL', disabled=False, icon='circle')
url_btn.on_click(open_url)
url_setup = VBox([url_text, url_btn])

input_setup = Accordion([cam_setup, url_setup, vid_up_box])
[input_setup.set_title(idx, title) for idx, title in enumerate(['Camera', 'URL', 'Video File'])]

# playback
cap_frame_btn = ToggleButton(description='Preview', disable=False, icon='camera')
start_btn = ToggleButton(description='Start', disable=False, icon='play')
frame_skip_spn = BoundedIntText(value=0, min=0, max=10, step=1, description='Frame skip', disabled=False)
# pb_ctrl = VBox([Label('Playback Control'), HBox([cap_frame_btn, start_btn]), frame_skip_spn])

cap_frame_btn.observe(cap_frame, names='value')
start_btn.observe(start_infer, names='value')

# main
image_widget = Image(format='jpeg')
# is_red_chk = Checkbox(value=False, description='Red Light', disabled=False, indent=False)
main_box = VBox([HBox([cap_frame_btn, start_btn, frame_skip_spn, save_img_out]), image_widget])

# status bar
clr_btn = Button(description='Clear Log', disable=False, icon='minus-square')
status_bar = Output()
status_box = VBox([clr_btn, status_bar])
clr_btn.on_click(lambda obj: status_bar.clear_output())

# left sidebar
left_roi = Accordion([model_setup, input_setup, roi_ctrl, line_ctrl])
[left_roi.set_title(idx, title) for idx, title in enumerate(['Model Setup', 'Input Setup', 'ROI Control', 'Stop Line Control'])]

# layout
app_layout = AppLayout(
  header=None,
  left_sidebar=left_roi,
  center=main_box,
  right_sidebar=out_box,
  footer=None,
)

apps = VBox([app_layout, status_box])


tt = time.perf_counter()
h, w = 480, 640
frame = np.zeros((h, w, 3), dtype=np.uint8)
image_widget.value = bgr8_to_jpeg(frame)

In [ ]:
display(apps)
# display(status_box)
# display(left_accordion)

In [ ]:
# to stop run all
assert False

In [ ]:
load_engines(load_engine_btn)
open_url(url_btn)

# Test Area

In [ ]:
# %%time
tt = time.perf_counter()
fps_filt = 0

frame = st.stream.read()
img = frame.copy()
# image_widget.value = bgr8_to_jpeg(img)

# roi shape
h_img, w_img = img.shape[:2]
verts = np.vstack((np.hstack([x.value for x in roi_xs])*w_img, np.hstack([y.value for y in roi_ys])*h_img)).T.astype(int)
verts[[2, 3]] = verts[[3,2]]
l,t,w,h = cv2.boundingRect(verts)

# stop line position
pos = np.hstack((np.array(stop_line_x.value)*w_img, np.array(stop_line_y.value)*h_img)).astype(int)
pos[[1, 2]] = pos[[2, 1]]

# get roi image
mask = np.zeros(img.shape, dtype=np.uint8)
cv2.fillPoly(mask, [np.int32(verts)], (255,)*img.shape[2])
masked = cv2.bitwise_and(img, mask)
roi = masked[t:t+h, l:l+w, :]

# infer car and windshield
cw_res = engine.infer(roi, imgsz=480, classes=[0, 3], render=False)

# check if any object is detected
if cw_res.xyxy.size>0:
  # restore box coordinate from roi coordinate to image coordinate
  cw_boxes = cw_res.xyxy
  expand_boxes(cw_boxes[:, :4], (l,t))
  cw_res.raw_img = frame
  cw_res.xyxy = cw_boxes

  # split the boxes separately
  cars = cw_res.get_class_boxes(0, 'xyxy')
  ws = cw_res.get_class_boxes(3, 'xyxy')

  # check if it detected the cars
  if cars.size > 0:
    # isolate the bounding box
    cboxes, cconfs, ccls_id = cars[:, :4], cars[:, 4], cars[:, 5]
    # create placeholder columns for other properties (id, stop_line, n_passenger, n_seat_belt)
    cboxes_id = np.zeros((cboxes.shape[0], cboxes.shape[1]+4))
    # get object id (tracker)
    cboxes_id[:, :5] = car_tracker.update(cboxes)
    ws_imgs = []

    # check if it detected the windshields
    if ws.size > 0:
      # isolate the bounding boxes
      wboxes, wconfs, wcls_id = ws[:, :4], ws[:, 4], ws[:, 5]
      # create placeholder column for id
      wboxes_id = -np.ones((wboxes.shape[0], wboxes.shape[1]+1))
      wboxes_id[:, :-1] = wboxes

      # assign the each windshield its car_id (based on iou)
      invalid = []
      for idx, w in enumerate(wboxes_id): #for each windshield bounding box
        # calculate its iou against all car bounding boxes
        car_index = bbox_iou(np.expand_dims(w[:4], 0), cars[:, :4])
        # windshield with no car will be discarded
        if car_index.sum()==0:
          invalid.append(idx)
          continue
        # assign the car_id with highest iou score
        w[-1] = cboxes_id[np.argmax(car_index), 4]
      # discard invalid boxes
      wboxes_id = np.delete(wboxes_id, invalid, 0)

      # seat belt inference
      ws_imgs = []
      for wbox, wconf, wcls_id in zip(wboxes_id, wconfs, wcls_id.astype(int)): # for each windshield bounding box
        box = wbox.astype(int)
        # isolate (crop) windshield from image
        ws = frame[box[1]:box[3], box[0]:box[2], :]
        # do the inference
        ps_res = engine.infer(ws, imgsz=480, classes=[1, 2], render=False)
        ws_imgs.append((int(wbox[-1]), ps_res))
        a = cboxes_id[cboxes_id[:, 4]==wbox[-1]]
        # count the detected seatbelt
        a[0, 7] = ps_res.classids[ps_res.classids==2].size
        a[0, 6] = ps_res.classids[ps_res.classids==1].size
        # append the result for that specific car_id
        cboxes_id[cboxes_id[:, 4]==wbox[-1]] = a

    # check if the cars crossed the stop line when the light is red
    if traffic_light.value=='Red':
      cboxes_id[:, 5] = is_crossing_stop_line(cboxes_id, pos).astype(cboxes_id.dtype)

    # update the result
    records = recorder.update(frame_id, cboxes_id, ws_imgs, cw_res)
    # draw result on image and result table
    rows = ""
    for idx in reversed(list(records.keys())):
      recs = records[idx]
      rows += row_template.format(idx, any(recs['stop_line']), max(recs['n_passenger']), max(recs['n_seat_belt']))
      if idx not in cboxes_id[:, 4]:
        continue
      plot_one_box(recs['positions'][-1], img, color=[0,0,255] if any(recs['stop_line']) else [84,185,29], label=f"id:{idx}, sb:{max(recs['n_seat_belt'])}")
    out_table.value = table_style + out_table_template.format(rows)

# draw stop line and roi
draw_stop_line(img, pos, stop_line_thickness.value, hex2bgr("#00ff00" if traffic_light.value=="Green" else "#ff0000"))
draw_roi(img, verts, roi_thickness.value, hex2bgr(roi_color.value))

# calculte fps and draw it on image
frame_id += 1
ed = time.perf_counter()
fps = 1/(ed-tt)
fps_filt = .9*fps_filt + .1*fps
tt = ed
img = imresize(img, height=480)
cv2.putText(img, f"FPS: {fps_filt:#.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
image_widget.value = bgr8_to_jpeg(img)
tt = ed

In [ ]:
od['reverse']

In [ ]:
for idx, recs in records.items():
  print(idx, end=' ')
  print(any(recs['stop_line']))

In [ ]:
hab = Image(format='jpeg')
display(hab)

In [ ]:
hab.value = bgr8_to_jpeg(records[0]['car_imgs'][3])

In [ ]:
print(cboxes_id.shape)
print(len(ws_imgs))
print(len(c_imgs))

In [ ]:
import pandas as pd
import json

In [ ]:
for idx, obj in records.items():
  for k, v in obj.items():
    if isinstance(v, list):
      for i, item in enumerate(v):
        records[idx][k][i] = item.tolist()

In [ ]:
with open('hab.json', 'w') as f:
  json.dump(records, f, indent=4)

In [ ]:
hub = pd.DataFrame.from_dict(records, orient='index')

In [ ]:
hub

In [ ]:
%load_ext line_profiler
%lprun -f run run()

In [ ]:
cw_res.render()
image_widget.value = bgr8_to_jpeg(cw_res.img)

In [ ]:
cw_res.yolo

In [ ]:
cw_res.pandas(form='xywh')

In [ ]:
cw_res.pandas(form='yolo')

In [ ]:
cw_res.yolo[:,:4]

In [ ]:
h